# CS295/395: Secure Distributed Computation
## Homework 3

## Definitions

In [2]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import numpy as np
import galois
GF = galois.GF(2 ** 31 - 1)

class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self):
        """Initialize the field size and dictionary to hold received messages."""
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

# Generate Shamir shares for secret v with threshold t and number of shares n
def shamir_share(v, t, n):
    coefficients = GF([GF.Random() for _ in range(t-1)] + [v])
    poly = galois.Poly(coefficients)
    shares = [(GF(x), poly(GF(x))) for x in range(1, n+1)]
    return shares

# Reconstruct the secret from at least t Shamir shares
def reconstruct(shares):
    xs = GF([s[0] for s in shares])
    ys = GF([s[1] for s in shares])
    poly = galois.lagrange_poly(xs, ys)
    #print(poly)
    secret = poly(0)
    
    return secret

# Sum up a list of shares, to get a share of the sum
def sum_shares(shares):
    xs = [s[0] for s in shares]
    ys = [s[1] for s in shares]
    
    # make sure all the xs are the same
    assert xs.count(xs[0]) == len(xs)
    
    # build the share and output
    return (xs[0], GF(ys).sum())

In [3]:
reconstruct(shamir_share(250, 3, 5))

GF(250, order=2147483647)

## Question 1 (20 points)

Implement a protocol that calculates the *mean* of the parties' input numbers, secure against semi-honest adversaries. The specification for the `mean` function is below.

In [4]:
nums = [np.random.randint(0, 200) for _ in range(10)]

def mean(nums):
    s = np.sum(nums)
    return s/len(nums)

mean(nums)

120.2

### Question 1(a)

Write an English description of the protocol, in the same format as the protocol definition from our exercise on 9/11. The ideal functionality is as follows:

\begin{equation*}
\textbf{Functionality: Mean}\\
\fbox{$\mathcal{F}_{mean}(x_1, \dots, x_n) = \frac{1}{n} \sum_{i=1}^n x_i$}
\end{equation*}

*HINT*: think about how to implement `mean` with addition *only*. Keep in mind that the number of parties is public knowledge (so it could be used to post-process the protocol's result).

**Protocol: Secure Summation with Additive Secret Sharing**
- **Round 1**: Each party $P_i$ sends one share of its input $x_i$ to each other party, keeping one share for itself.
- **Round 2**: Each party $P_i$ sums the shares it holds (including both the shares it has received and the share it kept for itself). Each party sends its sum to all other parties.
- **Opening**: Each party adds up the sums it receives and the sum it computed and then divides by the quantity of sums recieved. The quantity of sums recieves represents the number of parties which is equal to $n$.

### Question 1(b)

Implement your protocol from question 1(a), using Shamir secret sharing.

In [5]:
class MeanParty(Party):
    def round1(self, parties, input_num):
        self.input = GF(input_num)
        self.parties = parties
        n = len(parties)
        t = n-1
        
        shares = shamir_share(self.input, t, n)

        for share, party in zip(shares, parties):
            self.send(party, 1, share)

    def round2(self):
        baseline_x_coord = self.received[1][0][0]
        cummulative_y_coord = GF(0)

        for xcoord, ycoord in self.received[1]:
            assert xcoord == baseline_x_coord
            cummulative_y_coord += ycoord

        for party in self.parties:
            self.send(party, 2, (baseline_x_coord, cummulative_y_coord))

    
    def round3(self):
        # reconstruct the sum
        sum_share = reconstruct(self.received[2])

        self.output = int(sum_share) / len(self.parties)

In [6]:
# TEST CASE for question 1(b)

NUM_PARTIES = 10

parties = [MeanParty() for _ in range(NUM_PARTIES)]

nums = [np.random.randint(0, 10) for _ in range(NUM_PARTIES)]

for p,n in zip(parties, nums):
    p.round1(parties, n)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for i, p in enumerate(parties):
    #print(p.get_view())
    print(f'Output of party {i}: {p.output}')
    assert np.abs(p.output - np.mean(nums)) < 1
print(f'True answer: {np.mean(nums)}')

Output of party 0: 5.0
Output of party 1: 5.0
Output of party 2: 5.0
Output of party 3: 5.0
Output of party 4: 5.0
Output of party 5: 5.0
Output of party 6: 5.0
Output of party 7: 5.0
Output of party 8: 5.0
Output of party 9: 5.0
True answer: 5.0


## Question 2 (20 points)

Implement a protocol that calculates the *variance* of the parties' input numbers, secure against semi-honest adversaries. The specification for the `variance` function is below. Your protocol **may leak the mean of the numbers**.

In [7]:
def variance(nums):
    m = mean(nums)
    sq = [(x - m)**2 for x in nums]
    ssum = np.sum(sq)
    return ssum/len(nums)

variance(nums)

7.6

### Question 2(a)

Write an English description of the protocol, in the same format as the protocol definition from our exercise on 9/11.

\begin{equation*}
\textbf{Functionality: Variance}\\
\fbox{$\mathcal{F}_{var}(x_1, \dots, x_n) = \frac{\sum_{i=1}^n (x_i - \mu)^2}{n}$}\\
\text{where}\;\; \mu = \frac{1}{n} \sum_{i=1}^n x_i
\end{equation*}

*HINT*: Calculating the variance requires calculating the mean first. Start with your answer to 1(a), and extend it to calculate the variance with additional rounds. These rounds only require addition. As in question 1, the number of parties is public knowledge (so it could be used to post-process the protocol's result).

$\frac{\sum_{i=1}^n (x_i - \mu)^2}{n} = \frac{\sum_{i=1}^n x_i^2}{n} - \mu^2$

**Protocol: Secure Summation with Additive Secret Sharing**
- **Round 1**: Each party $P_i$ sends one share of its input $x_i$ to each other party, keeping one share for itself.
- **Round 2**: Each party $P_i$ sums the shares it holds (including both the shares it has received and the share it kept for itself). Each party sends its sum to all other parties.
- **Round 3**: Each party adds up the sums it receives and the sum it computed and then divides by the quantity of sums recieved. The quantity of sums recieves represents the number of parties which is equal to $n$. This is stored as $\mu$ for later. Create shares for $x_i^2$ and then distribute to all parties including self
- **Round 4** Each party $P_i$ sums the new shares it holds (including both the shares it has received and the share it kept for itself). Each party sends its sum to all other parties.
- **Round 5** Each party adds up the sums it receives and the sum it computed and then divides by the quantity of sums recieved. The quantity of sums recieves represents the number of parties which is equal to $n$. Then it subtracts $\mu^2$ from it and that is the variance



### Question 2(b)

Implement your protocol from question 2(a), using Shamir secret sharing. *HINT*: start with your solution to 1(b) and extend it.

In [8]:
class VarianceParty(Party):
    def round1(self, parties, input_num):
        self.input = GF(input_num)
        self.parties = parties
        n = len(parties)
        t = n-1
        
        self.shares = shamir_share(self.input, t, n)
        self.shares_squared = shamir_share(self.input**2, t, n)

        for share, party in zip(self.shares, parties):
            self.send(party, 1, share)

    def round2(self):
        baseline_x_coord = self.received[1][0][0]
        cummulative_y_coord = GF(0)

        for xcoord, ycoord in self.received[1]:
            assert xcoord == baseline_x_coord
            cummulative_y_coord += ycoord

        for party in self.parties:
            self.send(party, 2, (baseline_x_coord, cummulative_y_coord))

    
    def round3(self):
        # reconstruct the sum
        self.xi_sum = reconstruct(self.received[2])

        # remember the mean for next step
        self.mu = int(self.xi_sum) / len(self.parties)

        # distribute shares of x^2
        for share, party in zip(self.shares_squared, self.parties):
            self.send(party, 3, share)
            
    
    def round4(self):
        baseline_x_coord = self.received[3][0][0]
        cummulative_y_coord = GF(0)

        for xcoord, ycoord in self.received[3]:
            assert xcoord == baseline_x_coord
            cummulative_y_coord += ycoord

        for party in self.parties:
            self.send(party, 4, (baseline_x_coord, cummulative_y_coord))

    def round5(self):
        self.xi_squared_sum = reconstruct(self.received[4])

        self.output = int(self.xi_squared_sum) / len(self.parties) - self.mu**2

In [9]:
# TEST CASE for question 2(b)
NUM_PARTIES = 10

parties = [VarianceParty() for _ in range(NUM_PARTIES)]

nums = [np.random.randint(0, 10) for _ in range(NUM_PARTIES)]

for p,n in zip(parties, nums):
    p.round1(parties, n)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for p in parties:
    p.round4()
for p in parties:
    p.round5()
for i, p in enumerate(parties):
    #print(p.get_view())
    print(f'Output of party {i}: {p.output}')
    assert np.abs(p.output - np.var(nums)) <= 1
print(f'True answer: {np.var(nums)}')

Output of party 0: 4.809999999999995
Output of party 1: 4.809999999999995
Output of party 2: 4.809999999999995
Output of party 3: 4.809999999999995
Output of party 4: 4.809999999999995
Output of party 5: 4.809999999999995
Output of party 6: 4.809999999999995
Output of party 7: 4.809999999999995
Output of party 8: 4.809999999999995
Output of party 9: 4.809999999999995
True answer: 4.8100000000000005


## Question 3 (20 points)

Implement a protocol to *multiply* three input numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive one share (see the test case for details). The threshold $t$ will be set to $1/3$ of the number of parties $n$. The protocol is defined as follows:

\begin{equation*}
\textbf{Functionality: Multiply Three Numbers}\\
\fbox{$\mathcal{F}(a, b, c) = a \cdot b \cdot c$}
\end{equation*}


**Protocol: Multiplication with Shamir Secret Sharing**
- **Round 1**: Each party $P_i$ receives shares $a_i, b_i, c_i$ as input. $P_i$ performs finite field multiplication to calculate $s_i = a_i \times b_i \times c_i$, a share of the quantity $a\cdot b \cdot c$. $P_i$ sends $s_i$ to all of the parties.
- **Round 2**: Each party $P_i$ receives shares $s_1, \dots, s_n$ of the product. $P_i$ reconstructs the value from these shares and outputs the value.

In [20]:
class MultThreeParty(Party):
    def round1(self, parties, a_shr, b_shr, c_shr):
        self.input = (a_shr, b_shr, c_shr)
        assert a_shr[0] == b_shr[0] and b_shr[0] == c_shr[0]
        
        s_i_x = GF(a_shr[0])
        s_i_y = GF(a_shr[1] * b_shr[1] * c_shr[1])

        s_i = GF([s_i_x, s_i_y])

        for party in parties:
            self.send(party, 1, s_i)

    def round2(self):
        product = reconstruct(self.received[1])

        self.output = product

In [21]:
NUM_PARTIES = 10
# (t, n)-Shamir scheme
n = NUM_PARTIES
t = 3

shares1 = shamir_share(GF(5), t, n)
shares2 = shamir_share(GF(6), t, n)
shares3 = shamir_share(GF(7), t, n)

parties = [MultThreeParty() for _ in range(NUM_PARTIES)]

for p,s1,s2,s3 in zip(parties, shares1, shares2, shares3):
    p.round1(parties, s1, s2, s3)
for p in parties:
    p.round2()
for i, p in enumerate(parties):
    # print(p.get_view())
    print(f'Output of party {i}: {p.output}')
    assert p.output == 210

Output of party 0: 210
Output of party 1: 210
Output of party 2: 210
Output of party 3: 210
Output of party 4: 210
Output of party 5: 210
Output of party 6: 210
Output of party 7: 210
Output of party 8: 210
Output of party 9: 210


## Question 4 (10 points)

In 2-5 sentences, answer the following:

- How must we set the values of $t$ and $n$ for this protocol to work?
- Why does the protocol have this limitation?

We must set $t$ such that $t^3 <= n$. This is because we are not using degree reduction so every time we multiply the shares, we increase the degree of the polynomial, effectively increasing the threshold $t$. Because we are multiplying 3 numbers, we end up with an effective threshold of $t*t*t = t^3$. This threshold must be less than or equal to n so that we can actually decode the answer, and so $t^3 <= n$.